In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug  5 03:06:49 2023

@author: Administrator
"""

import math
from math import pow
import numpy as np
from scipy.optimize import least_squares

# =============================================================================
# Create functions for WiFi Positioning

# Simplified path loss model
def calc_dist(rssi,a,n):
    cal_d= pow(10,((rssi-a)/(-10*n)))
    return cal_d

# Calculate distance error
def dist_error_2d(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist 

# Least square algorithm for indoor positioning
def trilaterate(distances, positions):
  def residuals(x, distances, positions):
    residuals = []
    for i in range(len(distances)):
      residuals.append(np.linalg.norm(x - positions[i]) - distances[i])
    return residuals
  
  initial_guess = np.mean(positions, axis=0)
  result = least_squares(residuals, initial_guess, args=(distances, positions))
  return result.x

# 2D trilateration algorithm 
def trilateration_2d_co(distances, positions, name):
    # Calculate the position of the object
    position = trilaterate(distances, positions)
    
    final = [name, position[0], position[1]]
    return final

# 3D trilateration algorithm
def trilateration_3d_co(distances, positions, name):
    # Calculate the position of the object
    position = trilaterate(distances, positions)

    final = [name, position[0], position[1], position[2]]
    return final

In [4]:
# Create monitoring APs coordinate dictionary from xlsx
import pandas as pd 
import ast

df = pd.read_excel('Coordinate.xlsx')

f = df['Floor'].tolist()
loc_iy = df.values.tolist()
loc_iy = [[value for value in inner_list[1:] if not (isinstance(value, float) and math.isnan(value))] for inner_list in loc_iy]

# Convert list of lists of strings to a dictionary of lists of tuples
loc_iy = {str(f[i]): [ast.literal_eval(item) for item in inner_list] for i, inner_list in enumerate(loc_iy)}

# Set path loss exponent (n) and reference power (a)
n=3 # Path loss exponent
a=-37 # Reference power for 2.4 GHz
b=-49 # Reference power for 5 GHz

In [5]:
calc_dist(-55,a,n)

3.9810717055349722

In [6]:
distances = (4, 5, 6)
positions = ((1,1), (1,3), (5,5))
name = 'Nam'

trilateration_2d_co(distances, positions, name)

['Nam', 4.370163413567932, -0.9076335956243211]